In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
from rdkit import Chem
import pandas as pd
import numpy as np
from molmass import Formula
from matplotlib import pyplot as plt

import pickle
from pandas import DataFrame as df
from rdkit.Chem.Draw import IPythonConsole
import ast
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit.Chem.inchi import MolToInchi
from rdkit.Chem.rdmolfiles import MolFromSmiles

import xml.etree.ElementTree as ET

In [3]:
hmdb = pd.read_pickle('db/hmdb_out_molecule')

### Identifying a "core metabolome"

Rationale:  Searching against all of HMDB is problematic, due to the large number of drugs, exogenous metabolites, and theoretical metabolites.  As well, many human specific metabolites may not be expected over the diverse datasets in METASPACE.

The goal of this notebook, is to identify a "core metabolome" with reasonable coverage across species, a good chance of biological relevance, and a high-likelihood of being observed in MS experiments.

-Components:
1. Intersection of HMDB,* ECMDB,* YMDB* == core metabolites.
2. KEGG == core metabolites.*
3. Lipid Maps* == core lipids.
4. MSMLS (CMBR stds)* == likelihood of biological relevance since available.
5. Veronika's standards* == likelihood of biological relevance since available.
6. HMDB* with evidence of existence (observed/quantitated) == detectability.
7. Mayo clinic targeted* == biological/medical relevance.
*=DL'd data
https://docs.google.com/spreadsheets/d/1o0RMxNtcweox-zwEV2Qdvj9ErKrzFxTtCW5zhZTz1dE/edit?usp=sharing

-Done:
1. Download DB's.

-To do:
2. Parse to Pandas.
2. Compare across DB's as RDkit object.
3. Write final DB with ascension numbers.
4. Output to Vitally for custom DB on beta.

In [6]:
# Export: http://www.hmdb.ca/metabolites?utf8=%E2%9C%93&quantified=1&detected=1&filter=true
hmdb_detected = list(pd.read_csv('db/hmdb_detected', sep=',').HMDB_ID)
hmdb_det_df = hmdb[hmdb.id.isin(hmdb_detected)].copy(deep=True)

In [21]:
def can_smiles(smiles):
    try:
        return Chem.MolToSmiles(Chem.MolFromSmiles(smiles),True)
    except:
        return np.nan

In [24]:
ecmdb_df = pd.read_json('db/ecmdb.json')
ecmdb_df['can_smiles'] = ecmdb_df['moldb_smiles'].apply(lambda x: can_smiles(x))
ecmdb_df = ecmdb_df.dropna(subset=['can_smiles'])

RDKit ERROR: [14:44:09] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [14:44:09] Explicit valence for atom # 10 N, 4, is greater than permitted
RDKit ERROR: [14:44:09] Explicit valence for atom # 31 N, 4, is greater than permitted
RDKit WARNING: [14:44:09] WARNING: not removing hydrogen atom without neighbors


In [27]:
# Directly comparing smiles overlap between both databases!
h = hmdb.Smiles
e = ecmdb_df.can_smiles #
he_core = list(set(h).intersection(e))
len(list(set(h).intersection(e)))

795

In [50]:
from rdkit.Chem.PandasTools import LoadSDF
ymdb_df = LoadSDF('db/ymdb.sdf', smilesName='smiles')
ymdb_df['can_smiles'] = ymdb_df['SMILES'].apply(lambda x: can_smiles(x))
ymdb_df = ymdb_df.dropna(subset=['can_smiles'])
ymdb_list = list(ymdb_df.can_smiles)
hey_core = list(set(he_core).intersection(ymdb_list))
hmdb_hey_df = hmdb[hmdb.Smiles.isin(hey_core)].copy(deep=True)

RDKit ERROR: [14:55:20] ERROR: Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10 11 12 13 14
RDKit ERROR: 
RDKit ERROR: [14:56:57] Explicit valence for atom # 28 N, 4, is greater than permitted
RDKit ERROR: [14:56:57] ERROR: Could not sanitize molecule ending on line 5686
RDKit WARNING: [14:56:57] WARNING: not removing hydrogen atom without neighbors
RDKit ERROR: [14:56:57] ERROR: Explicit valence for atom # 28 N, 4, is greater than permitted
RDKit ERROR: [14:56:57] Explicit valence for atom # 28 N, 4, is greater than permitted
RDKit ERROR: [14:56:57] ERROR: Could not sanitize molecule ending on line 118027
RDKit ERROR: [14:56:57] ERROR: Explicit valence for atom # 28 N, 4, is greater than permitted
RDKit ERROR: [14:56:57] Explicit valence for atom # 21 N, 4, is greater than permitted
RDKit ERROR: [14:56:57] ERROR: Could not sanitize molecule ending on line 203103
RDKit ERROR: [14:56:57] ERROR: Explicit valence for atom # 21 N, 4, is greater than permitted
RDKit ERROR: [14:56:57] Can'

In [156]:
kegg_ids = []
with open('db/kegg_bio_cmpds.json', 'r') as k_file:
    for k_line in k_file:
        if ':"C' in k_line:
            var = k_line.split(':"')[1].split('  ')[0]
            kegg_ids.append(var)
        else:
            continue
temp = kegg_ids
for k in temp:
    if len(k) != 6:
        kegg_ids.remove(k)
        
# Serach KEGG ID's in http://cts.fiehnlab.ucdavis.edu/batch to INCHI

str(kegg_ids).replace("\'",'')
for k in kegg_ids:
    pass
    #print(k)
    
kegg_df = pd.DataFrame()
for k in ['db/KEGG_part1.csv', 'db/KEGG_part2.csv', 'db/KEGG_part3.csv']:
    temp_df = pd.read_csv(k) 
    kegg_df = pd.concat([kegg_df,temp_df])
    
kegg_hmdbs = kegg_df[kegg_df['Human Metabolome Database'] != 'No result'].copy(deep=True)
bad_kegg_df = kegg_df[kegg_df['Human Metabolome Database'] == 'No result'].copy(deep=True)

kegg_hmdbs_list = list(kegg_hmdbs['Human Metabolome Database'])

In [137]:
def can_smiles_finchi(inchi):
    try:
        return Chem.MolToSmiles(Chem.MolFromInchi(inchi),True)
    except:
        return np.nan

In [157]:
kegg_structs = bad_kegg_df[bad_kegg_df.InChIKey != 'undefined'].copy(deep=True)
# Offline on chemical trasnlation service to Inchi

kegg_struct_df = pd.read_csv('db/KEGG_part_4.csv')
kegg_struct_df = kegg_struct_df.drop(columns=['From','To']).rename(columns={'Term':'kegg', 'Result':'inchi'}).copy(deep=True)
kegg_struct_df['can_smiles'] = kegg_struct_df['inchi'].apply(lambda x: can_smiles_finchi(x))

kegg_struct_list = list(kegg_struct_df['can_smiles'])
kegg_hmdbs_2 = list(hmdb[hmdb.Smiles.isin(kegg_struct_list)].copy(deep=True).id)

kegg_hmdb_df = hmdb[hmdb.id.isin(kegg_hmdbs_list + kegg_hmdbs_2)].copy(deep=True)

kegg_not_hmdb = kegg_struct_df[~kegg_struct_df['can_smiles'].isin(list(kegg_hmdb_df.Smiles))]


In [160]:
# Lipid maps:
lm_df = LoadSDF('db/LMSD_20191002.sdf', smilesName='smiles')
lm_df['can_smiles'] = lm_df['SMILES'].apply(lambda x: can_smiles(x))
lm_df = lm_df.dropna(subset=['can_smiles'])
lm_list = list(lm_df.can_smiles)
hmdb_lm_df = hmdb[hmdb.Smiles.isin(lm_list)].copy(deep=True)

RDKit WARNING: [16:52:21] Warning: conflicting stereochemistry at atom 29 ignored.
RDKit WARNING: [16:52:21] Warning: conflicting stereochemistry at atom 29 ignored.
RDKit WARNING: [16:52:21] Warning: conflicting stereochemistry at atom 29 ignored.
RDKit WARNING: [16:52:21] Warning: conflicting stereochemistry at atom 29 ignored.
RDKit WARNING: [16:52:47] Warning: conflicting stereochemistry at atom 28 ignored.
RDKit WARNING: [16:52:47] Warning: conflicting stereochemistry at atom 28 ignored.
RDKit WARNING: [16:52:47] Warning: conflicting stereochemistry at atom 28 ignored.
RDKit WARNING: [16:52:47] Warning: conflicting stereochemistry at atom 28 ignored.
RDKit WARNING: [16:52:48] Warning: conflicting stereochemistry at atom 3 ignored.
RDKit WARNING: [16:52:48] Warning: conflicting stereochemistry at atom 3 ignored.
RDKit WARNING: [16:52:48] Warning: conflicting stereochemistry at atom 3 ignored.
RDKit WARNING: [16:52:48] Warning: conflicting stereochemistry at atom 3 ignored.
RDKit WA

In [177]:
# MSMLS (CMRBR stds)
msmls = pd.read_csv('db/MSMLS.tsv', sep='\t')
msmls['can_smiles'] = msmls['smiles'].apply(lambda x: can_smiles(x))
msmls = msmls.dropna(subset=['can_smiles'])
msmls_list = list(msmls.can_smiles)
hmdb_ms_df = hmdb[hmdb.Smiles.isin(msmls_list)].copy(deep=True)
msmls_not_hm = msmls[~msmls.can_smiles.isin(hmdb.Smiles)]

RDKit ERROR: [17:10:18] Explicit valence for atom # 15 N, 4, is greater than permitted


In [6]:
# Veronika's standards and Mayo clinic:
manual = pd.read_csv('db/core_metabolites.tsv', sep='\t')
hmdb_man = list(man_hmdb.HDMB.unique())[1:]
hmdb_man_df = hmdb[hmdb.id.isin(hmdb_man)].copy(deep=True)

In [33]:
not_hmdb = manual.dropna(subset=['SMILES']).copy(deep=True)

In [34]:
# Save your work:
hmdb_hey_df.to_pickle('df_pickles/hmdb_hey_df')
kegg_hmdb_df.to_pickle('df_pickles/kegg_hmdb_df')
kegg_not_hmdb.to_pickle('df_pickles/kegg_not_hmdb')
hmdb_lm_df.to_pickle('df_pickles/hmdb_lm_df')
hmdb_ms_df.to_pickle('df_pickles/hmdb_ms_df')
msmls_not_hm.to_pickle('df_pickles/msmls_not_hm')
hmdb_man_df.to_pickle('df_pickles/hmdb_man_df')
not_hmdb.to_pickle('df_pickles/not_hmdb')
hmdb_det_df.to_pickle('df_pickles/hmdb_det_df')

In [13]:
# 1. Intersection of HMDB,* ECMDB,* YMDB*
hmdb_hey_df: 333

# 2. KEGG*
kegg_hmdb_df:306
kegg_not_hmdb: 166 # To parse custom db

# 3. Lipid Maps*
hmdb_lm_df: 6316

# 4. MSMLS (CMBR stds)*
hmdb_ms_df: 324
msmls_not_hm: 204

# 5. Veronika's standards* 
# 7. Mayo clinic targeted*
hmdb_man_df: 153
not_hmdb: 4  # To parse custom db

# 6. HMDB* with evidence of existence
hmdb_det_df: 9008

In [36]:
# Load and join everything
core_df = pd.DataFrame()
df_list = ['hmdb_hey_df', 'kegg_hmdb_df', 'hmdb_lm_df', 
           'hmdb_ms_df', 'hmdb_man_df', 'hmdb_det_df']
for df in df_list:
    df = pd.read_pickle('df_pickles/' + df)
    core_df = pd.concat([core_df, df])
    
core_df.drop_duplicates(keep=False,inplace=True)
core_df.to_pickle('core_metabolome_df.pickle')
core_df['formula'] = core_df['Molecule'].apply(lambda x: CalcMolFormula(x))
cols = ['id', 'mol_name', 'formula', 'inchi'] 
core_df = core_df[cols].rename(columns={'mol_name': 'name'}).copy(deep=True)

In [38]:
new_db_list = ['df_pickles/kegg_not_hmdb', #166
               'df_pickles/msmls_not_hm', #210
               'df_pickles/not_hmdb'] #4

In [124]:
df = pd.read_pickle(new_db_list[0])
df['formula'] = df['can_smiles'].apply(lambda x: CalcMolFormula(Chem.MolFromSmiles(x)))
df = df.reset_index()
df = df.rename(columns={'kegg':'id'}).copy(deep=True)

In [125]:
colnames=['id', 'name']
kegg_names = pd.read_csv('db/br08001.keg', sep='@', names=colnames)
kegg_names.id = kegg_names.id.str.strip()
df = df.merge(kegg_names, left_on='id', right_on='id', how='left')
df1 = df[['id', 'name', 'formula', 'inchi']].drop_duplicates().copy(deep=True)

In [162]:
df = pd.read_pickle(new_db_list[1])
df = df.rename(columns={'cmpd':'name'}).copy(deep=True)
df['index1'] = df.index
df['id'] = df['index1'].apply(lambda x: 'msmls' + str(x))
df['inchi'] = df['can_smiles'].apply(lambda x: MolToInchi(MolFromSmiles(x)))
df2 = df[['id', 'name', 'formula', 'inchi']].drop_duplicates().copy(deep=True)

       id                    name  formula  \
2  msmls2  1-HYDROXY-2-NAPHTHOATE  C11H8O3   
5  msmls5    1-METHYLNICOTINAMIDE  C7H9N2O   

                                               inchi  
2  InChI=1S/C11H8O3/c12-10-8-4-2-1-3-7(8)5-6-9(10...  
5  InChI=1S/C7H8N2O/c1-9-4-2-3-6(5-9)7(8)10/h2-5H...  


In [172]:
df = pd.read_pickle(new_db_list[2])
df['inchi'] = df['SMILES'].apply(lambda x: MolToInchi(MolFromSmiles(x)))
df = df.rename(columns={'Name':'name', 'PubChem CID:': 'id',
                       'Formula': 'formula'}).copy(deep=True)
df3 = df[['id', 'name', 'formula', 'inchi']].drop_duplicates().copy(deep=True)
df3

,id,name,formula,inchi
0,68067,"1,8-diaminonaphtalene",C10H10N2,InChI=1S/C10H10N2/c11-8-5-1-3-7-4-2-6-9(12)10(...
1,3469,"2,5-dihydroxybenzoic acid",C7H6O4,InChI=1S/C7H6O4/c8-4-1-2-6(9)5(3-4)7(10)11/h1-...
2,7019,9-aminoacridine,C13H10N2,InChI=1S/C13H10N2/c14-13-9-5-1-3-7-11(9)15-12-...
73,11966124,C14-CoA,C35H62N7O17P3S,InChI=1S/C35H62N7O17P3S/c1-4-5-6-7-8-9-10-11-1...


In [189]:
df = pd.concat([df1, df2])
df = pd.concat([df, df3])
core_df = pd.concat([df, core_df])
core_df = core_df.dropna().copy(deep=True)
core_df = core_df.drop_duplicates().copy(deep=True)
core_metabolome.to_pickle('core_metabolome_v1.pickle')
core_metabolome.to_csv('core_metabolome_v1.txt', sep='\t', index=False)
core_df.shape

(13965, 4)

In [3]:
core_metabolome = pd.read_pickle('core_metabolome_v1.pickle')

In [4]:
def neutral(test):
    if '-' in test:
        x = test.split('-')[0]
        if 'H' not in x:
            prefix = x
            final_suffix = 'H'
        else:       
            x = x.split('H')
            prefix = x[0]
            suffix = x[1]
            if suffix[0].isdigit() is False:
                # CHX
                final_suffix = 'H2' + suffix
            else:
                if suffix[1].isdigit() is False:
                    # CHnX
                    n = int(suffix[0])
                    n +=1
                    s_list = list(suffix)[1:]          
                    final_suffix = 'H' + str(n) + "".join(s_list)
                elif suffix[2].isdigit() is False:
                    # CHnnX
                    s_list = list(suffix)
                    n = int(str(s_list[0]) + str(s_list[1]))
                    n += 1
                    s_list = list(suffix)[2:]
                    final_suffix = 'H' + str(n) + "".join(s_list)
                else:
                    # CHnnnX
                    s_list = list(suffix)
                    n = int(str(s_list[0]) + str(s_list[1]) + str(s_list[2]))
                    n += 1
                    s_list = list(suffix)[3:]
                    final_suffix = 'H' + str(n) + "".join(s_list)
        return (prefix + final_suffix)
    elif '+' in test and 'H' in test:
        x = test.split('+')[0]
        x = x.split('H')
        prefix = x[0]
        suffix = x[1]
        if len(suffix) is 0:
            return prefix + 'H2'
        
        if suffix[0].isdigit() is False:
            # CH2X
            final_suffix = suffix
        else:
            if suffix[1].isdigit() is False:
                # CHnX
                n = int(suffix[0])
                n -=1
                s_list = list(suffix)[1:]          
                final_suffix = 'H' + str(n) + "".join(s_list)
            elif suffix[2].isdigit() is False:
                # CHnnX
                s_list = list(suffix)
                n = int(str(s_list[0]) + str(s_list[1]))
                n -= 1
                s_list = list(suffix)[2:]
                final_suffix = 'H' + str(n) + "".join(s_list)
            else:
                # CHnnnX
                s_list = list(suffix)
                n = int(str(s_list[0]) + str(s_list[1]) + str(s_list[2]))
                n -= 1
                s_list = list(suffix)[3:]
                final_suffix = 'H' + str(n) + "".join(s_list)
        return (prefix + final_suffix)
    else:
        return test

In [5]:
core_metabolome['formula'] = core_metabolome['formula'].apply(lambda x:
                                                             neutral(x))

In [7]:
core_metabolome.formula.str.contains('-').sum() # 9 negative, all salts, okay to drop.

0

In [15]:
core_metabolome.formula.str.contains('\+').sum() # 94 positives

0

In [13]:
metals = list(core_metabolome[core_metabolome.formula.str.contains('\+')].name)
core_metabolome = core_metabolome[~core_metabolome.name.isin(metals)].copy(deep=True)

In [47]:
core_metabolome.name.sample(10)

15129                                     5-Deoxykievitone
14930                                  2'-Hydroxygenistein
41806                                    Thiocarbamic acid
161                             7alpha-Hydroxytestosterone
2810                          3-Sialyl-N-acetyllactosamine
902                                          Pyruvaldehyde
8492                                       12(13)Ep-9-KODE
4090                  PC(18:0/22:6(4Z,7Z,10Z,13Z,16Z,19Z))
34753    TG(22:2(13Z,16Z)/16:1(9Z)/22:6(4Z,7Z,10Z,13Z,1...
34148    TG(20:3n6/20:3(5Z,8Z,11Z)/22:6(4Z,7Z,10Z,13Z,1...
Name: name, dtype: object

In [49]:
core_metabolome.to_pickle('core_metabolome_v2.pickle')
core_metabolome.to_csv('core_metabolome_v2.txt', sep='\t', index=False)
core_metabolome.to_csv('/Users/dis/PycharmProjects/neutral_loss/core_metabolome_v2.txt', sep='\t', index=False)